In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
stockdata_all = pd.read_csv('stockdata.csv', encoding = 'cp950')

In [3]:
stocklist = [2317]

In [4]:
stockdata_raw = stockdata_all[stockdata_all['code']==stocklist[0]][['code','date','close']]

In [5]:
stockdata_raw.head()

,code,date,close
755037,2317,20070102,58.44
755038,2317,20070103,58.69
755039,2317,20070104,59.43
755040,2317,20070105,57.83
755041,2317,20070108,57.09


In [7]:
#===============================initial parameters===========================
Mm = 10000000
cash_ratio = 0.2
M = Mm*(1-cash_ratio)
Time = 3
day_in_year = 252
r = 0.013
h = 0.5
vol_day = 20
in_sample_last_year = 2012
tc_buy = 0.001425
tc_sell = 0.004425

In [8]:
stockdata = stockdata_raw
stockdata['lag_close'] = stockdata['close'].shift(1)
stockdata['return'] = np.log(stockdata['close']/stockdata['lag_close'])
stockdata['vol'] = math.sqrt(252)*stockdata['return'].rolling(vol_day).std()

In [9]:
stockdata.head()

,code,date,close,lag_close,return,vol
755037,2317,20070102,58.44,NaN,NaN,NaN
755038,2317,20070103,58.69,58.44,0.004269,NaN
755039,2317,20070104,59.43,58.69,0.012530,NaN
755040,2317,20070105,57.83,59.43,-0.027291,NaN
755041,2317,20070108,57.09,57.83,-0.012879,NaN


In [10]:
in_sample_stock_data = stockdata[stockdata['date']<=(in_sample_last_year*10000+1231)]

In [11]:
in_sample_stock_data.tail()

,code,date,close,lag_close,return,vol
756527,2317,20121224,52.42,52.60,-0.003428,0.275596
756528,2317,20121225,53.01,52.42,0.011192,0.278622
756529,2317,20121226,52.84,53.01,-0.003212,0.278621
756530,2317,20121227,53.13,52.84,0.005473,0.279599
756531,2317,20121228,53.07,53.13,-0.001130,0.271165


In [12]:
out_stock_data = stockdata[stockdata['date']>(in_sample_last_year*10000+1231)]

In [13]:
in_time_sp = in_sample_stock_data['close'].quantile(0.2)

In [14]:
in_position = np.amin(np.where(out_stock_data['close'] <= in_time_sp))

In [ ]:
if np.isinf(in_position)==False :
    out_stock_data1 = out_stock_data[in_position:]
    out_stock_data1 = out_stock_data1[0:(Time*day_in_year)]
    S = out_stock_data1['close'][0:1]
    vol=out_stock_data1['vol'][0:1]


In [ ]:
#================================Rcode(約在R程式碼第49、50行附近)========================================

myFunction <- function(x){   # 設定求解方程
      y <- numeric(2)
      m <- x[1] 
      K <- x[2]
      d1 <- (log(S/K)+(r+0.5*vol^2)*Time)/(vol*sqrt(Time))
      d2 <- d1-vol*sqrt(Time)
      y[1] <- ( M*(1+h)/K-m )
      y[2] <- (M-m*(S*pnorm(-d1)+K*exp(-r*Time)*pnorm(d2)) )
      return(y)
    }